In [22]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
#from plotly.offline import init_notebook_mode, iplot
#import plotly.graph_objs as go
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [384]:
##### 파일 불러오기 #####
df = pd.read_csv("C:\\Users\\asus\\BigContest\\files\\data_preprocessing\\1_food_waste_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'waste_cnt', 'waste_amt']  ##### 각자 데이터에 맞게 열이름 변경하기 #####
df

,base_date,emd_cd,waste_cnt,waste_amt
base_date,,,,
2019-11-30,2019-11-30,50110250,620,2279550
2019-12-31,2019-12-31,50110250,8178,31493650
2020-01-31,2020-01-31,50110250,13042,49632850
2020-02-29,2020-02-29,50110250,12264,46197050
2020-03-31,2020-03-31,50110250,14316,52561750
...,...,...,...,...
2021-02-28,2021-02-28,50130620,4563,15566250
2021-03-31,2021-03-31,50130620,4920,17300600
2021-04-30,2021-04-30,50130620,5256,17646850


In [385]:
# 필요 없는 열 제거
data = df.drop(['base_date', 'waste_amt'], 1)     ##### 각자 필요없는 열 제거 #####
data

,emd_cd,waste_cnt
base_date,,
2019-11-30,50110250,620
2019-12-31,50110250,8178
2020-01-31,50110250,13042
2020-02-29,50110250,12264
2020-03-31,50110250,14316
...,...,...
2021-02-28,50130620,4563
2021-03-31,50130620,4920
2021-04-30,50130620,5256


In [386]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50130590  ##### 각 행정동코드 입력 #####

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,waste_cnt
base_date,
2018-01-31,21666
2018-02-28,19252
2018-03-31,21567
2018-04-30,21424
2018-05-31,22382
2018-06-30,22734
2018-07-31,26787
2018-08-31,26246
2018-09-30,23877


In [387]:
# series 형으로 변환
sub_df.to_dict('series')

{'waste_cnt': base_date
 2018-01-31    21666
 2018-02-28    19252
 2018-03-31    21567
 2018-04-30    21424
 2018-05-31    22382
 2018-06-30    22734
 2018-07-31    26787
 2018-08-31    26246
 2018-09-30    23877
 2018-10-31    22438
 2018-11-30    21689
 2018-12-31    21606
 2019-01-31    20265
 2019-02-28    17234
 2019-03-31    15630
 2019-04-30    14778
 2019-05-31    19665
 2019-06-30    21529
 2019-07-31    27006
 2019-08-31    27051
 2019-09-30    24430
 2019-10-31    23606
 2019-11-30    22345
 2019-12-31    21921
 2020-01-31    21953
 2020-02-29    21164
 2020-03-31    23648
 2020-04-30    22566
 2020-05-31    24730
 2020-06-30    23663
 2020-07-31    21926
 2020-08-31    21378
 2020-09-30    18609
 2020-10-31    17296
 2020-11-30    15613
 2020-12-31    17194
 2021-01-31    17589
 2021-02-28    16256
 2021-03-31    17931
 2021-04-30    18317
 2021-05-31    20159
 2021-06-30    20790
 Name: waste_cnt, dtype: int64}

# ARIMA

In [388]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)
#sub_df_float.tail()

In [389]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))
    
##### 여기서 p-value 확인 -> 0.05보다 크면 아래로 내려가서 차분하기 / 작으면 차분 생략(아래 셀 생략) #####

ADF Statistic : -3.408121
p-value : 0.010685
Critical Values :
	1%: -3.610400
	5%: -2.939109
	10%: -2.608063


In [390]:
##### 이건 설치하는 거라 처음 돌릴 때 한 번만 주석 풀어서 설치하고 그 이후로는 필요없음! #####

#!pip install pmdarima

In [391]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

##### 여기서 아래쪽에 'Best model:  ARIMA(0,1,0)(0,0,0)[1] intercept' 이렇게 나오면 (0,1,0) 가져다 쓸거임 #####
##### ARIMA(0,1,0) 여기 숫자 세 개만 잘 보면 됨 !!!! #####

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=798.662, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=773.317, Time=0.05 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=755.605, Time=0.12 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=755.560, Time=0.13 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=754.731, Time=0.10 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=inf, Time=0.20 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=758.018, Time=0.01 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=758.180, Time=0.02 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=754.963, Time=0.03 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=756.238, Time=0.04 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=756.970, Time=0.08 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=756.111, Time=0.03 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=758.186, Time=0.13 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=755.964, Time=0.05 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=757.318, Time=0.09 sec
 ARIMA(3,0,0)(0,0,0)[1] inter

ARIMA(order=(3, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [392]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.waste_cnt.values, order=(3,0,0))     ##### 여기 order=(0,1,0)에 위에서 나온 숫자 넣는거 #####
                                    ##### 그리고 sub_df.card_amt.values에서 가운데 card_amt 각자걸로 바꾸기 #####

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\Users\asus\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(3, 0)   Log Likelihood                -371.435
Method:                       css-mle   S.D. of innovations           1646.572
Date:                Fri, 10 Sep 2021   AIC                            752.870
Time:                        15:44:29   BIC                            761.558
Sample:                             0   HQIC                           756.055
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.126e+04    723.905     29.362      0.000    1.98e+04    2.27e+04
ar.L1.y        0.9117      0.144      6.313      0.000       0.629       1.195
ar.L2.y        0.0888      0.212      0.420      0.6

# 예측하기

In [393]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

##### 맨 앞에 나온 'array([10681271.81014634, 10741037.02029268])' 이게 차례대로 7, 8월 예측값임 #####
##### 7월 : 10681271.81014634, 8월 : 10741037.02029268 #####
##### 이 값 가져다 엑셀파일에 입력하기 #####

(array([21765.28532004, 22063.87054531]), array([1646.57188941, 2228.1775977 ]), array([[18538.06371885, 24992.50692123],
       [17696.72270266, 26431.01838796]]))
